In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [2]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "sigmoid")(top_model)
    return top_model

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'roshan/train/'
validation_data_dir = 'roshan/validation/'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 16
img_rows=224
img_cols=224
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("family_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 400
nb_validation_samples = 100
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("family_vgg.h5")


Epoch 1/5
25/25 [==============================] - 386s 15s/step - loss: 1.1430 - accuracy: 0.7126 - val_loss: 5.3990e-04 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00054, saving model to family_vgg.h5
Epoch 2/5
25/25 [==============================] - 371s 15s/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 1.8340e-04 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.00054 to 0.00018, saving model to family_vgg.h5
Epoch 3/5
25/25 [==============================] - 365s 15s/step - loss: 4.4188e-04 - accuracy: 1.0000 - val_loss: 5.9798e-07 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.00018 to 0.00000, saving model to family_vgg.h5
Epoch 4/5
25/25 [==============================] - 373s 15s/step - loss: 0.3579 - accuracy: 0.9756 - val_loss: 8.6017e-05 - val_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/5
25/25 [==============================] - 2503s 100s/step - loss: 3.1120e-04 - accuracy: 1.0000 - 

In [6]:
from keras.models import load_model
model = load_model('family_vgg.h5')

In [11]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):

    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (224, 224))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    image, face = face_detector(frame)

    face=np.array(face)
    face=np.expand_dims(face,axis=0)
    if face.shape==(1,0):
        cv2.putText(image,"I don't know", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        cv2.imshow('Face Recognition',image)
    else:
        result=model.predict(face)
        print(result)
        if result[0][0] == 0.0:
            cv2.putText(image,"hey ", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition',image)
        else:
            cv2.putText(image,"Not recognized", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition',image)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

[[0. 1.]]
[[0. 1.]]
[[1.235573e-33 1.000000e+00]]
[[0. 1.]]
[[1.19842185e-26 1.00000000e+00]]
[[9.9998081e-01 2.9885233e-04]]
[[6.6675086e-32 1.0000000e+00]]
[[0. 1.]]
[[2.7838362e-16 1.0000000e+00]]
[[9.561052e-23 1.000000e+00]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[1.1854787e-38 1.0000000e+00]]
[[6.948093e-26 1.000000e+00]]
[[1.0016725e-37 1.0000000e+00]]
[[1.2228546e-21 1.0000000e+00]]
[[1.0000000e+00 1.3204509e-07]]
[[3.6944524e-28 1.0000000e+00]]
[[7.423332e-36 1.000000e+00]]
[[1.5917864e-34 1.0000000e+00]]
[[4.8282e-31 1.0000e+00]]
[[0. 1.]]
[[3.867823e-37 1.000000e+00]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[6.9767033e-13 1.0000000e+00]]
[[0. 1.]]
[[1.6331885e-33 1.0000000e+00]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[9.463137e-34 1.000000e+00]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[0. 1.]]
[[4.6370596e-30 1.0000000e+00]]
[[1.7594905e-32 1.0000000e+00]]
[[1.7697979e-38 1.0000000e+00]]
[[0. 1.]]
[[